# Calculate Vacancy Formation Energy in Silicon using MatterSim Potential

This notebook demonstrates how to compute the vacancy formation energy in silicon using the MatterSim potential within the Mat3ra framework.

## Methodology

The vacancy formation energy is calculated using the following approach:

1. **Create pristine supercell**: Build a silicon supercell from the unit cell and relax it using MatterSim potential
2. **Create vacancy**: Remove a single atom from the center of the supercell
3. **Relax vacancy structure**: Optimize the geometry of the defective supercell
4. **Calculate formation energy**: Compute the energy difference according to the formula:
   
   E_formation = E_vacancy - (N_vacancy/N_bulk) × E_bulk
   
   where:
   - E_vacancy: Total energy of the supercell with vacancy
   - E_bulk: Total energy of the pristine supercell
   - N_vacancy: Number of atoms in vacancy supercell
   - N_bulk: Number of atoms in pristine supercell

In [ ]:
import json

# Install required packages if not already installed
# INFO: if not installed correctly, clear environment and run `pip install .[forcefields]` in the terminal
try:
    import mattersim
except ImportError:
    import subprocess, sys
    subprocess.run([sys.executable, "-m", "pip", "install", ".[forcefields]", "--quiet"], check=False)
    import mattersim

## 1. Prepare materials
### 1.1. Load material data

In [ ]:
from ase.io import read
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material
from mat3ra.made.tools.convert import from_ase
import json
FOLDER = "data"
FILE_NAME = "generated_crystals.extxyz"
atoms = read(f"{FOLDER}/{FILE_NAME}")

# source_json = json.loads(source)
original_material = Material.create(from_ase(atoms))

### 1.2. Add modifications to material
Use any of the tools to modify the material, e.g. create supercell, add vacancy, add interstitials, etc.
Below is the example of creating a supercell and adding a vacancy.

In [ ]:
# from mat3ra.made.tools.helpers import create_vacancy, create_supercell

# supercell = create_supercell(original_material, scaling_factor=[2,2,2])
# material = create_vacancy(supercell, coordinate=[0.5, 0.5, 0.5], placement_method="closest_site")

## 1.3. Visualize materials

In [ ]:
from utils.visualize import visualize_materials

visualize_materials([material], rotation="-90x,-90y")

## 2. Setup calculation
### 2.1. Convert to ASE atoms

In [ ]:
from mat3ra.made.tools.convert import to_ase

original_material_atoms =  to_ase(original_material)
material_atoms = to_ase(material)

### 2.2. Setup MatterSim calculator

In [ ]:
from mattersim.forcefield.potential import MatterSimCalculator
from mattersim.applications.relax import Relaxer

original_material_atoms.calc = MatterSimCalculator()
material_atoms.calc = MatterSimCalculator()

### 2.3. Relax structures

In [ ]:
relaxer = Relaxer(optimizer="BFGS", constrain_symmetry=True)
relaxer.relax(material_atoms, steps=500)  # In-place relaxation


### 2.4. Visualize relaxed structures

In [ ]:
from mat3ra.made.tools.convert import from_ase

relaxed_material = Material.create(from_ase(material_atoms))

visualize_materials([original_material, relaxed_material], rotation="-90x,-90y")

## 3. Calculate energy differences

### 3.1. Get energies and atom counts

In [ ]:
energy_relaxed = material_atoms.get_potential_energy()
energy_original = original_material_atoms.get_potential_energy()

### 3.2. Calculate energy difference

In [ ]:
e_difference = energy_relaxed - energy_original
print(f"Energy difference: {e_difference} eV")

n_original = len(original_material_atoms)
n_relaxed = len(material_atoms)

energy_original_per_atom = energy_original / n_original
energy_difference_per_atom = e_difference / n_relaxed

print(f"Energy difference per atom: {energy_difference_per_atom} eV/atom")
